In [1]:
import numpy as np
import pandas as pd 
import xgboost as xgb 
import pickle
from datetime import datetime

import warnings

warnings.filterwarnings('ignore')

## Created Test data with it's actual reorder product

In [3]:
## I used test data and find their actual reorder product and I will use this for test purpose.

def cmb_prd(x):
    return " ".join(list(x.astype('str')))

def actual_test_label():
    test = pd.read_csv('Data/sample_submission.csv/sample_submission.csv',usecols=['order_id'])
    orders = pd.read_csv('Data/orders.csv/orders.csv',usecols=['order_id','user_id'])

    test = pd.merge(test,orders,on='order_id',how='inner')
    del orders
    
    prior = pd.read_csv('Data/order_products__prior.csv/order_products__prior.csv',usecols=['order_id','reordered','product_id'])

    orders = pd.read_csv('Data/orders.csv/orders.csv',usecols=['order_id','user_id'])

    p_o = pd.merge(orders,prior,on='order_id',how='inner')

    del prior,orders
    
    test = pd.merge(test,p_o,on='user_id',how='inner')
    
    del p_o
    
    test.drop(['order_id_y'],axis=1,inplace=True)

    test.rename(columns={'order_id_x':'order_id'},inplace=True)
    
    test = test[test['reordered']==1]
    
    actual_label = test.groupby("order_id")["product_id"].aggregate(cmb_prd).reset_index()
    actual_label.columns = ["order_id", "actual_products"]
    
    del test
    
    actual_label.to_pickle('test_data_label.pkl')
    

In [4]:
actual_test_label()

### Fun1 for predicting the reorder product for given Order ID

<ul><li>Here you can pass only one Order ID</li></ul>
    

In [89]:
def fun1(order_id):
    
    orders = pd.read_csv('Data/orders.csv/orders.csv',usecols=['order_id','user_id'])

    actual_test_data = pd.read_pickle('test_data_label.pkl')
    test = pd.merge(actual_test_data,orders,on='order_id',how='inner')

    del orders
    
    feat_data = pd.read_pickle('prior_final.pkl')

    test = pd.merge(test,feat_data,on='user_id',how='inner')

    del feat_data,actual_test_data
    
    query = test[test['order_id']==order_id]
    
    query = query[['product_id','usr_prd_count','usr_prd_sum','usr_latest']]
    
    model = pickle.load(open('xgboost.sav','rb'))
    
    test_data = xgb.DMatrix(np.array(query))

    pred = model.predict(test_data)
    
    
    cutoff = 0.2
    pred[pred>=cutoff] = 1
    pred[pred<cutoff] = 0
    query['pred'] = pred
    
    query = query[query['pred']==1]

    prd = " ".join(list(query['product_id'].astype('str')))
    if len(prd)==0:
        prd= None

    final = pd.DataFrame({
        'order_id':[order_id],
        'pred':[prd]
    })
    
    return final

### Fun2 Print the F1-score for actual reorder basket and predicted reorder basket

In [131]:
# https://www.kaggle.com/aikinogard/python-f1-score-function

def fun2(inp):
    
    actual_test_data = pd.read_pickle('test_data_label.pkl')

    actual = actual_test_data[actual_test_data['order_id']==inp]

    if len(actual) == 0:
        print('The Order ID does not exist')

    pred = pred_ans[pred_ans['order_id']==inp]

    a = actual['actual_products'].to_list()
    a = a[0].split(' ')

    b = pred['pred'].to_list()
    b = b[0].split(' ')

    y_true = set(a)
    y_pred = set(b)

    cross_size = len(y_true) & len(y_pred)
    if cross_size == 0: 
        print('No products in the list')
    else:
        p = 1. * cross_size / len(y_pred)
        r = 1. * cross_size / len(y_true)
        print(f'The Mean F1 Score of this basket is :{round(2 * p * r / (p + r),3)}')

In [137]:
start = datetime.now()

inp = int(input('Enter the order_id: '))

pred_ans = fun1(inp)

fun2(inp)
print(f'Time Taken:{datetime.now()-start}')

Enter the order_id: 137
The Mean F1 Score of this basket is :0.762
Time Taken:0:00:05.417720
